In [1]:
%cd /code

/code


In [2]:
from pprint import pprint
from source.service.datasets import DATA
from source.library.scraping import scrape_all_urls#, scrape_urls
import source.config.config as config

# This is needed because openai.text_completion calls asynchronous functions but
# Jupyter is already running its own event loop.
import nest_asyncio
nest_asyncio.apply()

In [3]:
langchain_doc_chunks = DATA.langchain_doc_chunks.load()
print(len(langchain_doc_chunks))
print(langchain_doc_chunks[0])

12329
('https://python.langchain.com/en/latest', 'Welcome to LangChain # LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model, but will also be: Data-aware : connect a language model to other sources of data Agentic : allow a language model to interact with its environment The LangChain framework is designed')


In [4]:
from langchain.vectorstores import Chroma
persist_directory = '/code/.vectordb/'

In [5]:
vectordb = Chroma(persist_directory=persist_directory)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
vectordb.similarity_search(query="What is Langchain?", k=5)

[Document(page_content="Human: For LangChain! Have you heard of it? AI:  No, I haven't heard of LangChain. Can you tell me more about it? Human: Haha nope, although a lot of people confuse it for that AI: > Finished chain. ' Oh, okay. What is LangChain?'", metadata={'url': 'https://python.langchain.com/en/latest/modules/memory/types/summary_buffer.html'}),
 Document(page_content='such as OpenAI or Hugging Face. LangChain is a software development framework designed to simplify the creation of applications using large language models (LLMs). Written in: Python and JavaScript. Initial release: October 2022. LangChain - The A.I-native developer toolkit We started LangChain with the intent to build a modular and flexible framework for developing A.I- ... LangChain explained in', metadata={'url': 'https://python.langchain.com/en/latest/modules/chains/examples/flare.html'}),
 Document(page_content='of all related terms, papers, methods, etc. Whether implemented in LangChain or not!\\nGallery

---

In [7]:
# from langchain.chains import VectorDBQA
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp

# N_CONTEXT = 1024
# model_path = './models/ggml-alpaca-7b-q4.bin'

N_CONTEXT = 2048
model_path = config.LLM_VICUNA_13B

llm = LlamaCpp(model_path=model_path, temperature=0.0, n_ctx=N_CONTEXT)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    verbose=False,
)

llama.cpp: loading model from /llms/models/ggml-vic13b-q5_1.bin
llama_model_load_internal: format     = ggjt v2 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 9 (mostly Q5_1)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =  90.75 KB
llama_model_load_internal: mem required  = 11359.05 MB (+ 1608.00 MB per state)
llama_init_from_file: kv self size  = 1600.00 MB
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [8]:
pprint(qa.json())

('{"memory": null, "verbose": false, "combine_documents_chain": {"memory": '
 'null, "verbose": false, "input_key": "input_documents", "output_key": '
 '"output_text", "llm_chain": {"memory": null, "verbose": false, "prompt": '
 '{"input_variables": ["context", "question"], "output_parser": null, '
 '"partial_variables": {}, "template": "Use the following pieces of context to '
 "answer the question at the end. If you don't know the answer, just say that "
 "you don't know, don't try to make up an "
 'answer.\\n\\n{context}\\n\\nQuestion: {question}\\nHelpful Answer:", '
 '"template_format": "f-string", "validate_template": true, "_type": '
 '"prompt"}, "llm": {"model_path": "/llms/models/ggml-vic13b-q5_1.bin", '
 '"suffix": null, "max_tokens": 256, "temperature": 0.0, "top_p": 0.95, '
 '"logprobs": null, "echo": false, "stop_sequences": [], "repeat_penalty": '
 '1.1, "top_k": 40, "_type": "llama.cpp"}, "output_key": "text", "_type": '
 '"llm_chain"}, "document_prompt": {"input_variabl

In [9]:
qa.combine_documents_chain.llm_chain.prompt.template

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"

---

In [10]:
question = "What is LangChain?"
base_response = llm(prompt=f"Question: {question}? Answer: ")
base_response


llama_print_timings:        load time =  1626.76 ms
llama_print_timings:      sample time =    45.87 ms /   101 runs   (    0.45 ms per token)
llama_print_timings: prompt eval time =  2264.73 ms /    11 tokens (  205.88 ms per token)
llama_print_timings:        eval time = 32664.93 ms /   100 runs   (  326.65 ms per token)
llama_print_timings:       total time = 43718.69 ms


" LangChain is a new programming language that combines the power of Python, Java, C++, and JavaScript. It's designed to be easy to learn and use, with a syntax that's simple and intuitive. LangChain can be used for web development, data analysis, machine learning, and more. It's perfect for beginners and experienced programmers alike!  #LangChain #ProgrammingLanguage #PythonJavaC++JavaScript"

In [11]:
pprint(base_response)

(' LangChain is a new programming language that combines the power of Python, '
 "Java, C++, and JavaScript. It's designed to be easy to learn and use, with a "
 "syntax that's simple and intuitive. LangChain can be used for web "
 "development, data analysis, machine learning, and more. It's perfect for "
 'beginners and experienced programmers alike!  #LangChain '
 '#ProgrammingLanguage #PythonJavaC++JavaScript')


In [12]:
# print("Base-Model:")
# print(llm(prompt=f"Question: {question}? Answer: "))
print("\nContext-Model")
context_answer = qa(question)
print(context_answer)

Llama.generate: prefix-match hit



Context-Model
{'query': 'What is LangChain?', 'result': ' LangChain is a software development framework designed to simplify the creation of applications using large language models (LLMs). It is written in Python and JavaScript, and was initially released in October 2022. LangChain aims to provide a modular and flexible toolkit for developing AI-native applications, with a focus on making it easy for developers to build, test, and deploy LLM-based projects. The LangChain framework includes a number of features and tools, including a gallery of project examples, deployment instructions and templates, a Discord community for discussion, and resources for tracing and debugging.', 'source_documents': [Document(page_content="Human: For LangChain! Have you heard of it? AI:  No, I haven't heard of LangChain. Can you tell me more about it? Human: Haha nope, although a lot of people confuse it for that AI: > Finished chain. ' Oh, okay. What is LangChain?'", metadata={'url': 'https://python.la


llama_print_timings:        load time =  1626.76 ms
llama_print_timings:      sample time =    63.07 ms /   126 runs   (    0.50 ms per token)
llama_print_timings: prompt eval time = 78244.98 ms /   400 tokens (  195.61 ms per token)
llama_print_timings:        eval time = 48597.17 ms /   125 runs   (  388.78 ms per token)
llama_print_timings:       total time = 138900.16 ms


In [13]:
pprint(context_answer['result'])

(' LangChain is a software development framework designed to simplify the '
 'creation of applications using large language models (LLMs). It is written '
 'in Python and JavaScript, and was initially released in October 2022. '
 'LangChain aims to provide a modular and flexible toolkit for developing '
 'AI-native applications, with a focus on making it easy for developers to '
 'build, test, and deploy LLM-based projects. The LangChain framework includes '
 'a number of features and tools, including a gallery of project examples, '
 'deployment instructions and templates, a Discord community for discussion, '
 'and resources for tracing and debugging.')


In [14]:
vectordb.similarity_search(query="What is Langchain?", k=5)

[Document(page_content="Human: For LangChain! Have you heard of it? AI:  No, I haven't heard of LangChain. Can you tell me more about it? Human: Haha nope, although a lot of people confuse it for that AI: > Finished chain. ' Oh, okay. What is LangChain?'", metadata={'url': 'https://python.langchain.com/en/latest/modules/memory/types/summary_buffer.html'}),
 Document(page_content='such as OpenAI or Hugging Face. LangChain is a software development framework designed to simplify the creation of applications using large language models (LLMs). Written in: Python and JavaScript. Initial release: October 2022. LangChain - The A.I-native developer toolkit We started LangChain with the intent to build a modular and flexible framework for developing A.I- ... LangChain explained in', metadata={'url': 'https://python.langchain.com/en/latest/modules/chains/examples/flare.html'}),
 Document(page_content='of all related terms, papers, methods, etc. Whether implemented in LangChain or not!\\nGallery

---

In [15]:
question = "How do I use prompt templates for creating a chat bot?"
print("Base-Model:")
pprint(llm(prompt=f"Question: {question}? Answer: "))
print("\nContext-Model")
context_answer = qa(question)
pprint(context_answer['result'])

Base-Model:


Llama.generate: prefix-match hit

llama_print_timings:        load time =  1626.76 ms
llama_print_timings:      sample time =   117.29 ms /   256 runs   (    0.46 ms per token)
llama_print_timings: prompt eval time =  3114.97 ms /    16 tokens (  194.69 ms per token)
llama_print_timings:        eval time = 83151.58 ms /   256 runs   (  324.81 ms per token)
llama_print_timings:       total time = 108608.18 ms
Llama.generate: prefix-match hit


('\n'
 '\n'
 'To create a chatbot using prompt templates, you can follow these steps:\n'
 '\n'
 '1. Create a new file in your project directory and name it `prompts.json`.\n'
 '2. Define the prompts that your chatbot will ask the user. Each prompt '
 'should be an object with a `text` property for the message to display and a '
 '`value` property for the expected response from the user. For example:\n'
 '```perl\n'
 '{\n'
 '  "text": "What is your name?",\n'
 '  "value": "user_name"\n'
 '}\n'
 '```\n'
 '3. Save the `prompts.json` file.\n'
 '4. In your chatbot code, load the prompts from the `prompts.json` file using '
 'the `readFileSync` method:\n'
 '```javascript\n'
 "const fs = require('fs');\n"
 "const prompts = JSON.parse(fs.readFileSync('./prompts.json', 'utf8'));\n"
 '```\n'
 '5. Define a function to ask the user for their response to each prompt. This '
 "function should display the prompt, wait for the user's response, and then "
 'check if the response matches the `value` pro


llama_print_timings:        load time =  1626.76 ms
llama_print_timings:      sample time =    52.13 ms /   114 runs   (    0.46 ms per token)
llama_print_timings: prompt eval time = 71395.72 ms /   368 tokens (  194.01 ms per token)
llama_print_timings:        eval time = 38824.33 ms /   114 runs   (  340.56 ms per token)
llama_print_timings:       total time = 120333.10 ms


---

In [16]:
question = "How do I use prompt templates for creating a chat bot?"
print("Base-Model:")
base_answer = llm(prompt=f"Question: {question}? Answer: ")
pprint(base_answer)
print("\nContext-Model")
context_answer = qa(question)
pprint(context_answer['result'])

Llama.generate: prefix-match hit


Base-Model:
('\n'
 '\n'
 'To create a chatbot using prompt templates, you can follow these steps:\n'
 '\n'
 '1. Create a new file in your project directory and name it `prompts.json`.\n'
 '2. Define the prompts that your chatbot will ask the user. Each prompt '
 'should be an object with a `text` property for the message to display and a '
 '`value` property for the expected response from the user. For example:\n'
 '```perl\n'
 '{\n'
 '  "text": "What is your name?",\n'
 '  "value": "user_name"\n'
 '}\n'
 '```\n'
 '3. Save the `prompts.json` file.\n'
 '4. In your chatbot code, load the prompts from the `prompts.json` file using '
 'the `readFileSync` method:\n'
 '```javascript\n'
 "const fs = require('fs');\n"
 "const prompts = JSON.parse(fs.readFileSync('./prompts.json', 'utf8'));\n"
 '```\n'
 '5. Define a function to ask the user for their response to each prompt. This '
 "function should display the prompt, wait for the user's response, and then "
 'check if the response matches the


llama_print_timings:        load time =  1626.76 ms
llama_print_timings:      sample time =   116.70 ms /   256 runs   (    0.46 ms per token)
llama_print_timings: prompt eval time =  3037.80 ms /    16 tokens (  189.86 ms per token)
llama_print_timings:        eval time = 83086.60 ms /   256 runs   (  324.56 ms per token)
llama_print_timings:       total time = 108433.02 ms
Llama.generate: prefix-match hit


(' You can create a custom prompt template by using a ChatPromptTemplate and '
 'combining one or more MessagePromptTemplates. The ChatPromptTemplate allows '
 'you to format the prompt in a structured way, while the '
 'MessagePromptTemplates provide specific prompts for different types of '
 'messages. To use the custom prompt template, you can call the '
 '`format_prompt` method on the ChatPromptTemplate and pass in any necessary '
 'variables or data. The resulting PromptValue can then be converted to a '
 'string or Message object as needed.')



llama_print_timings:        load time =  1626.76 ms
llama_print_timings:      sample time =    52.00 ms /   114 runs   (    0.46 ms per token)
llama_print_timings: prompt eval time = 71560.73 ms /   368 tokens (  194.46 ms per token)
llama_print_timings:        eval time = 38451.99 ms /   114 runs   (  337.30 ms per token)
llama_print_timings:       total time = 120126.44 ms


In [17]:
results = vectordb.similarity_search(query=question, k=5)
for r in results:
    print(r.metadata['url'])
    print(r.page_content)

https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/custom_prompt_template.html
of default prompt templates here . Creating a Custom Prompt Template # There are essentially two distinct prompt templates available - string prompt templates and chat prompt templates. String prompt templates provides a simple prompt in string format, while chat prompt templates produces a more structured prompt to be used with a chat API. In this guide, we will create a custom prompt using a
https://python.langchain.com/en/latest/modules/prompts.html
retrying if necessary). Getting Started # Getting Started Go Deeper # Prompt Templates Chat Prompt Template Example Selectors Output Parsers
https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/figma.html
have no idea if the Jon Carmack thing makes for better code. YMMV. # See https://python.langchain.com/en/latest/modules/models/chat/getting_started.html for chat info system_prompt_template = """You ar

In [18]:
vectordb.similarity_search(query=base_answer, k=5)

[Document(page_content='of default prompt templates here . Creating a Custom Prompt Template # There are essentially two distinct prompt templates available - string prompt templates and chat prompt templates. String prompt templates provides a simple prompt in string format, while chat prompt templates produces a more structured prompt to be used with a chat API. In this guide, we will create a custom prompt using a', metadata={'url': 'https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/custom_prompt_template.html'}),
 Document(page_content='have no idea if the Jon Carmack thing makes for better code. YMMV. # See https://python.langchain.com/en/latest/modules/models/chat/getting_started.html for chat info system_prompt_template = """You are expert coder Jon Carmack. Use the provided design context to create idomatic HTML/CSS code as possible based on the user request. Everything must be inline in one file and your response must be', metadata={'url': 'https

In [19]:
vectordb.similarity_search(query=context_answer, k=5)

[Document(page_content='of default prompt templates here . Creating a Custom Prompt Template # There are essentially two distinct prompt templates available - string prompt templates and chat prompt templates. String prompt templates provides a simple prompt in string format, while chat prompt templates produces a more structured prompt to be used with a chat API. In this guide, we will create a custom prompt using a', metadata={'url': 'https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/custom_prompt_template.html'}),
 Document(page_content='retrying if necessary). Getting Started # Getting Started Go Deeper # Prompt Templates Chat Prompt Template Example Selectors Output Parsers', metadata={'url': 'https://python.langchain.com/en/latest/modules/prompts.html'}),
 Document(page_content='have no idea if the Jon Carmack thing makes for better code. YMMV. # See https://python.langchain.com/en/latest/modules/models/chat/getting_started.html for chat info system

In [20]:
question = "What is the map_reduce chain type?"
print("Base-Model:")
base_answer = llm(prompt=f"Question: {question}? Answer: ")
print(base_answer)
print("Context-Model:")
context_answer = qa(question)
print(context_answer)

Base-Model:


Llama.generate: prefix-match hit

llama_print_timings:        load time =  1626.76 ms
llama_print_timings:      sample time =    42.80 ms /    94 runs   (    0.46 ms per token)
llama_print_timings: prompt eval time =  2862.22 ms /    14 tokens (  204.44 ms per token)
llama_print_timings:        eval time = 29810.59 ms /    93 runs   (  320.54 ms per token)
llama_print_timings:       total time = 40799.23 ms
Llama.generate: prefix-match hit




The map\_reduce chain type is a data structure that represents a sequence of map and reduce operations on a dataset. It is used to represent the intermediate results of a MapReduce job, allowing multiple jobs to be chained together to form a complete workflow. The map\_reduce chain type is similar to the pipeline chain type in that it allows for the composition of data processing operations, but it is specifically designed for MapReduce workflows.
Context-Model:
{'query': 'What is the map_reduce chain type?', 'result': ' The map\\_reduce chain type is a specific type of chain in the langchain library that is used for summarization. It can be loaded using the load\\_summarize\\_chain function and is typically used with the OpenAIModerationChain class to pass input through a moderation endpoint before processing.', 'source_documents': [Document(page_content='una task force dedicata ai crimini degli oligarchi russi. Il Presidente ha anche annunciato l\'approvazione della legge bipartiti


llama_print_timings:        load time =  1626.76 ms
llama_print_timings:      sample time =    29.13 ms /    63 runs   (    0.46 ms per token)
llama_print_timings: prompt eval time = 93919.06 ms /   482 tokens (  194.85 ms per token)
llama_print_timings:        eval time = 21528.15 ms /    62 runs   (  347.23 ms per token)
llama_print_timings:       total time = 121196.83 ms


---

# Appendix

In [21]:
from bs4 import BeautifulSoup
import numpy as np
import chromadb

client = chromadb.Client()

collection = client.create_collection('sample_collection')

documents = []
metadatas = []

for url, chunk in langchain_doc_chunks:
        documents.append(chunk)
        metadatas.append({'url': url})

collection.add(
    documents=documents, # we embed for you, or bring your own
    metadatas=metadatas, # filter on arbitrary metadata!
    ids=[str(x) for x in range(len(documents))], # must be unique for each doc 
)

print(np.mean([len(x) for x in documents]))
print(np.max([len(x) for x in documents]))

383.0272528185579
400


In [22]:
print(documents[0:10])

['Welcome to LangChain # LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model, but will also be: Data-aware : connect a language model to other sources of data Agentic : allow a language model to interact with its environment The LangChain framework is designed', 'is designed around these principles. This is the Python specific portion of the documentation. For a purely conceptual guide to LangChain, see here . For the JavaScript documentation, see here . Getting Started # How to get started using LangChain to create an Language Model application. Quickstart Guide Concepts and terminology. Concepts and terminology Tutorials created by community experts and', 'experts and presented on YouTube. Tutorials Modules # These modules are the core abstractions which we view as the building blocks of any LLM-powered application. For each module LangChain provid

In [23]:
results = collection.query(
    query_texts="What is a document loader?",
    n_results=5,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)
print(len(results['documents'][0]))

5


In [24]:
results

{'ids': [['4289', '6653', '6956', '6858', '6655']],
 'embeddings': None,
 'documents': [[', the loader\nwill track additional metadata like the page number and text type (i.e. title, narrative text)\nwhen that information is available.',
   'the functionality we dive deep on those topics. For an overview of everything related to this, please see the below notebook for getting started: Getting Started We then provide a deep dive on the four main components. Document Loaders How to load documents from a variety of sources. Text Splitters An overview of the abstractions and implementions around splitting text. VectorStores An overview of',
   ". load () page_data [Document(page_content='Introduction to GitBook\\nGitBook is a modern documentation platform where teams can document everything from products to internal knowledge bases and APIs.\\nWe want to help \\nteams to work more efficiently\\n by creating a simple yet powerful platform for them to \\nshare their knowledge\\n.\\nOur missi